In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.1))

In [ ]:
order=3
fesc = VectorL2(mesh, order=order, covariant=True)
festr = TangentialFacetFESpace(mesh, order=order )
gfE = GridFunction(fesc)
gfH = GridFunction(fesc)

peak =  exp(-100 * ( (x-0.5)**2+(y-0.5)**2+(z-1)**2))
gfE.Set ( (peak, 0, 0))

In [ ]:
Draw (gfE, draw_surf=False)

In [ ]:
traceop = ConvertOperator (fesc, festr) # , geom_free=True)

gfEtr = GridFunction(festr)
gfEtr.vec.data = traceop * gfE.vec

In [ ]:
Draw (gfEtr, draw_vol=False)

In [ ]:
E,dE = fesc.TnT()
H,dH = fesc.TnT()
Ehat,dEhat = festr.TnT()

n = specialcf.normal(3)

Bel = BilinearForm(trialspace=fesc, testspace=fesc) # , geom_free = True)
Bel += curl(E)*dE * dx # + E*Cross(dH,n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=festr, testspace=fesc) # , geom_free = True)
# Btr += -Ehat * Cross(dH,n)*dx(element_boundary=True)
Btr.Assemble()

B = Bel.mat + Btr.mat @ traceop

In [ ]:
mass = BilinearForm(E*dE*dx).Assemble().mat
invmass = mass.Inverse()
# invmass = fesc.Mass(1).Inverse()


In [ ]:
# hv = gfE.vec.CreateVector()
# hv[:] = Bel.mat * gfE.vec
# print ("norm hv = ", Norm(hv))

gfH.vec[:] = invmass@B * gfE.vec
print ("norm E = ", Norm(gfE.vec))
print ("norm H = ", Norm(gfH.vec))
gfHtr = GridFunction(festr)
gfHtr.vec.data = traceop * gfH.vec

Draw (gfHtr, draw_vol=False)